In [ ]:
import xarray as xr
import glob
import os

def process_and_save_data(year_range, input_path, output_path):
    for year in year_range:
        # Get a list of all .nc files available in different folders
        filenames = glob.glob(os.path.join(input_path, f"CCMP_y{year}*.nc"))

        # Open and merge the datasets
        dsmerged = xr.open_mfdataset(filenames)
        
        # Select a subset of the data
        ds = dsmerged.sel(lats=slice(302, 342), lons=slice(1200, 1308)).drop_vars('nobs')

        # Fix duplicate dates issue caused by mfdataset
        _, index = ds['time'].to_index().unique(return_index=True)
        ds = ds.isel(time=index)

        # Save the processed dataset
        ds.to_netcdf(os.path.join(output_path, f'CCMP_y{year}.nc'))

        # Remove input files
        for file_path in filenames:
            try:
                os.remove(file_path)
            except Exception as e:
                print(f"Error removing file {file_path}: {e}")

if __name__ == "__main__":
    # Range de anos a ser processado
    target_years = range(2016, 2017)

    # Diretórios de entrada e saída
    input_directory = 'd:/00_Masters/99_testes/'
    output_directory = 'd:/00_Masters/01_Dados/CCMP/'

    # Chamar a função para processamento e salvamento de dados
    process_and_save_data(target_years, input_directory, output_directory)
